# DEMO: Iskanje po PubMed z uporabo E-utilities

Na primeru si bomo ogledali, kako iščemo pu zbirki PubMed prek programskega dostopa.

## BioPython
Uporabili bomo [**BioPython**](https://biopython.org), ki vsebuje orodja za delo z bio-podatki (zaporedja, strukture ipd.); podrobnejša navodila najdete na njegovi domači strani, še posebej uporabna je "kuharica" [Biopython Tutorial and Cookbook](http://biopython.org/DIST/docs/tutorial/Tutorial.html), kjer najdete različne primere uporabe z razlagami (težava je le v tem, da nekateri primeri iz kuharice ne delujejo, saj niso bili posodobljeni glede na novo verzijo BioPythona). To ne bo edina uporaba BioPythona - uporabljali ga bomo namreč tudi pri drugih vajah.

Del BioPythona, ki nam omogoča iskanje zbirkah na strežnikih NCBI, med drugim PubMed, se imenuje **Bio.Entrez**, zanj je v okviru BioPython na voljo [dokumentacija](https://biopython.org/docs/1.75/api/Bio.Entrez.html).

### Modul Bio.Entrez

Modul Bio.Entrez nudi več funkcij, nekaj jih je navedenih tukaj (ostale so opisane v dokumentaciji):
* *esearch* (Entrez Search) - z njim iščemo po zbirki/zbirkah, vrne nam pa primerne ključe za dostop (ID), ki se jih nato uporabi za prenos posmeznih zapisov z npr. *efetch*;
* *efetch* (Entrez Fetch) - zapise (kot vhod podamo ključe ID) prenesemo v zahtevanem formatu;
* *esummary* (Entrez Summary) - za zapise (ID) nam prenese njihove kratke opise;
* *read* - analizira rezultate v XML, ki nam jih vrnejo zgoraj navedene funkcije;
* *egquery* - za iskalni pojem nam vrne število zadetkov v posameznih zbirkah, do katerih dostopamo prek Entrez;
* *einfo* - vrne nam imena polij ipd.

Modul Bio.Entrez je dejansko osnovan na API (*application programming interface*) za Entrez, za katerega obstaja uradna dokumentacija na strežnikih NCBI:
* [A General Introduction to the E-utilities](https://www.ncbi.nlm.nih.gov/books/NBK25497/),
* [Entrez Programming Utilities Help](https://www.ncbi.nlm.nih.gov/books/NBK25501/).


**Opozorila**:
* Pri programskem dostopu do Entrez se morate identificirati z e-poštnim naslovom (v primerih spodaj, če jih boste poganjali, nastavite vrednost ```Èntrez.email``` na vaš naslov.
* Če boste poizvedbe izvajali z visoko frekvenco (npr. zelo veliko število poizvedb na minuto) vas lahko strežnik začasno blokira. Podobno velja za visoko frekvenco poizvedb s prenosom velike količine podatkov (npr. velikega števila celotnih zapisov).

---
## Zgledi

V nadaljevanju je prikazano nekaj zgledov, ki demonstrirajo poizvedovanje s programskih dostopom do Entrez.

Najprej so oglejmo, katere podatkovne zbirke so nam na voljo prek tega vmesnika. 

In [1]:
from Bio import Entrez
Entrez.email = 'sem.napisite.vas@e-postni.naslov'   # identificiramo se!
handle = Entrez.einfo()
record = Entrez.read(handle)
handle.close()
print(record)   # to nam izpiše vrednost, ki je slovar

{'DbList': ['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'biosystems', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']}


Vidimo, da je ```record``` zapisan kot slovar, vrednost ključa ```DbList``` (Database List, ki je dejansko seznam zbirk) pa dobimo tako:

In [2]:
print(record['DbList'])   # izpiše nam vrednost ključa DbList (Database List)

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'biosystems', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']


Na voljo imamo torej zbirko *pubmed* (literaturna zbirka), zbirko aminokislinskih (*protein*) in nukleotidnih zaporedij (*nucleotide*) , struktur (*structure) itd.

Na tem mestu se bomo osredotočili na iskanje po zbirku Pubmed, zato bomo vrednost ```db``` (database) ustrezno nastavili na *pubmed*. Definirati moramo tudi iskalni pojem (v spodnjem primeru ```search_term```) - slednjega lahko definiramo kar z navedbo ene ali več besed, s katerimi želimo iskati po PubMedu, lahko pa uporabimo naprednejšo sintakso in sicer na enak način kot prek naprednega spletnega vmesnika za iskanje po tej podatkovni zbirki, kar smo si ogledali [pri prejšnji vaji](pubmed_web.md). Če si želimo kompleksnejšo poizvedbo, si celoten iskalni pojem najenostavneje zgradimo tako, da gremo na [Pubmed Advanced Search Builder](https://pubmed.ncbi.nlm.nih.gov/advanced/), zgradimo iskanje, ter vsebino iskalne škatle (*query box*) skopiramo in prilepimo kot vrednost ```search_term```.

Pri iskanju moramo zraven zbirke in iskalnega pojma definirati še *retrieval mode* oz. ```retmode``` (včasih še *retrieval type* - ```rettype```), možne vrednosti so navedene v [pomoči](https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/).

Spodaj je en tak primer, kjer za iščemo po imenih člankov, zanimajo pa nas samo taki, kjer se v imenu (*Title*) pojavi *coronavirus*, kjerkoli pa mora biti zraven tega še beseda *Tasmania* (iskanje sicer ni oblutljivo na velike in male črke, to pomeni, da ni *case-sensitive*). Kot rezultat dobimo slovar, ki vsebuje število zadetkov (*count*), seznam številk PMID zapisov oz. člankov (*IDList*) in uporabljen iskalni pojem (kot ga interpretira PubMed, vključno z [MeSH](https://www.ncbi.nlm.nih.gov/mesh/)).

In [3]:
# štetje zapisov
from Bio import Entrez
Entrez.email = 'sem.napisite.vas@e-postni.naslov' # identificiramo se
search_term = '(coronavirus[Title]) AND Tasmania' # definiramo iskanje
handle = Entrez.esearch(db='pubmed', term=search_term, retmode='xml')
records = Entrez.read(handle)
handle.close()
print(records)

{'Count': '4', 'RetMax': '4', 'RetStart': '0', 'IdList': ['33254559', '33195319', '32826150', '32244852'], 'TranslationSet': [{'From': 'Tasmania', 'To': '"tasmania"[MeSH Terms] OR "tasmania"[All Fields]'}], 'TranslationStack': [{'Term': 'coronavirus[Title]', 'Field': 'Title', 'Count': '17168', 'Explode': 'N'}, {'Term': '"tasmania"[MeSH Terms]', 'Field': 'MeSH Terms', 'Count': '1514', 'Explode': 'Y'}, {'Term': '"tasmania"[All Fields]', 'Field': 'All Fields', 'Count': '14171', 'Explode': 'N'}, 'OR', 'GROUP', 'AND'], 'QueryTranslation': 'coronavirus[Title] AND ("tasmania"[MeSH Terms] OR "tasmania"[All Fields])'}


Če nas zanima število zadetkov si lahko to vrednost enostavno izpišemo. Ali koda pravilno deluje lahko preverimo tudi tako, da iskanje izvedemo z uporabo [brskalnika](https://pubmed.ncbi.nlm.nih.gov/?term=%28coronavirus%5BTitle%5D%29+AND+%28Tasmania%5BAbstract%5D%29).

In [4]:
count = int(records['Count']) # pretvorimo Count v celo število
print(count) # izpišemo število zadetkov

4


Kot vidimo po zbirki iščemo z uporabo *esearch*, če pa želimo prenesti zapise, pa uporabimo *efetch* in sicer v kombinaciji s seznamom PMID (v *IdList*).

Za primer si oglejmo, kako je sestavljen edem izmed zapisov. Spodnja koda vzame prvo kodo PMID v seznamu *IdList* ter prenese in izpiše vsebino zapisa:

In [5]:
first_record = records['IdList'][0]   # prvi zapis v seznamu
print('Prvi zapis je:', first_record)   # izpis kode PMID prvega zapisa v seznamu
one_pubmed_entry = Entrez.efetch(db="pubmed", id=first_record, retmode='xml')
single_record = Entrez.read(one_pubmed_entry)
print(single_record)   # izpis zapisa

Prvi zapis je: 33254559


{'PubmedArticle': [{'MedlineCitation': DictElement({'CitationSubset': ['IM'], 'KeywordList': [ListElement([StringElement('COVID-19', attributes={'MajorTopicYN': 'N'}), StringElement('Drug delivery', attributes={'MajorTopicYN': 'N'}), StringElement('Nanoparticles', attributes={'MajorTopicYN': 'N'})], attributes={'Owner': 'NOTNLM'})], 'SpaceFlightMission': [], 'GeneralNote': [], 'OtherAbstract': [], 'OtherID': [], 'PMID': StringElement('33254559', attributes={'Version': '1'}), 'DateCompleted': {'Year': '2021', 'Month': '01', 'Day': '05'}, 'DateRevised': {'Year': '2021', 'Month': '01', 'Day': '05'}, 'Article': DictElement({'ArticleDate': [DictElement({'Year': '2020', 'Month': '09', 'Day': '10'}, attributes={'DateType': 'Electronic'})], 'Language': ['eng'], 'ELocationID': [StringElement('S0306-9877(20)32396-3', attributes={'EIdType': 'pii', 'ValidYN': 'Y'}), StringElement('10.1016/j.mehy.2020.110254', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})], 'Journal': {'ISSN': StringElement('1532-

Vidimo, da je zapis kar ena *solata*, tako da se v njem malce težko znajti. Spodnja koda nam recimo za vsak PMID izpiše povzetek, če je ta na voljo (tu spet vzamemo vse zadetke prejšnjega iskanja), kar je že nekako lepše.

In [6]:
for identifier in records['IdList']:
    pubmed_entry = Entrez.efetch(db="pubmed", id=identifier, retmode="xml")
    result = Entrez.read(pubmed_entry)
    article = result['PubmedArticle'][0]['MedlineCitation']['Article']
    if 'Abstract' in article:
        print(article['Abstract']['AbstractText'][0])

The highly contagious coronavirus, which had already affected more than 2 million people in 210 countries, triggered a colossal economic crisis consequently resulting from measures adopted by various goverments to limit transmission. This has placed the lives of many people infected worldwide at great risk. Currently there are no established or validated treatments for COVID-19, that is approved worldwide. Nanocarriers may offer a wide range of applications that could be developed into risk-free approaches for successful therapeutic strategies that may lead to immunisation against the severe acute respiratory syndrome coronavirus 2 (SARS‑CoV‑2) which is the primary causative organism that had led to the current COVID-19 pandemic. We address existing as well as emerging therapeutic and prophylactic approaches that may enable us to effectively combat this pandemic, and also may help to identify the key areas where nano-scientists can step in.


Coronavirus disease 2019 (COVID-19) is a rapidly evolving, highly transmissible, and potentially lethal pandemic caused by a novel coronavirus, severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). As of June 11 2020, more than 7,000,000 COVID-19 cases have been reported worldwide, and more than 400,000 patients have died, affecting at least 188 countries. While literature on the disease is rapidly accumulating, an integrated, multinational perspective on clinical manifestations, immunological effects, diagnosis, prevention, and treatment of COVID-19 can be of global benefit. We aimed to synthesize the most relevant literature and experiences in different parts of the world through our global consortium of experts to provide a consensus-based document at this early stage of the pandemic.


The coronavirus disease 2019 (COVID-19) pandemic is challenging healthcare systems worldwide, none more so than critical and intensive care settings. Significant attention has been paid to the capacity of Australian intensive care unit (ICUs) to respond to a COVID-19 surge, particularly in relation to beds, ventilators, staffing, personal protective equipment, and unparalleled increase in deaths in ICUs associated with COVID-19 seen internationally. While death is not uncommon in critical care, the international experience demonstrates that restrictions to family presence at the end of life result in significant distress for families and clinicians. As a result, the Australian College of Critical Care Nurses and the Australasian College for Infection Prevention and Control supported the development of a position statement to provide critical care nurses with specific guidance and recommendations for practice for this emerging priority area. Where possible, position statements are found

The epicenter of the original outbreak in China has high male smoking rates of around 50%, and early reported death rates have an emphasis on older males, therefore the likelihood of smokers being overrepresented in fatalities is high. In Iran, China, Italy, and South Korea, female smoking rates are much lower than males. Fewer females have contracted the virus. If this analysis is correct, then Indonesia would be expected to begin experiencing high rates of Covid-19 because its male smoking rate is over 60% (Tobacco Atlas). Smokers are vulnerable to respiratory viruses. Smoking can upregulate angiotensin-converting enzyme-2 (ACE2) receptor, the known receptor for both the severe acute respiratory syndrome (SARS)-coronavirus (SARS-CoV) and the human respiratory coronavirus NL638. This could also be true for new electronic smoking devices such as electronic cigarettes and "heat-not-burn" IQOS devices. ACE2 could be a novel adhesion molecule for SARS-CoV-2 causing Covid-19 and a potentia

Rezultate lahko shranimo tudi lokalno, na primer v obliki datoteke CSV (*Comma Separated Values*). Pri tem uporabimo knjižnico [**Pandas**](https://pandas.pydata.org/), kar ilustrira spodnji zgled, ki:
* definira slovar *articles_with_abstracts*,
* za vsako kodo PMID prenese zapis ter ga doda v slovar pod ključem, ki je enak PMID,
* pretvori slovar v *Pandas dataframe*,
* shrani *dataframe* kot datoteko csv.

In [7]:
import pandas as pd
articles_with_abstracts = {}   # ustvarimo slovar

# spodnja koda je podobna kot zgoraj, le da ne izpisuje povzetkov
for identifier in records['IdList']:
    pubmed_entry = Entrez.efetch(db="pubmed", id=identifier, retmode="xml")
    results = Entrez.read(pubmed_entry)
    article = results['PubmedArticle'][0]['MedlineCitation']['Article']
    if 'Abstract' in article:
        articles_with_abstracts[identifier] = article['Abstract']['AbstractText'][0]

# print(articles_with_abstracts)   # to bi nam izpisalo slovar
df = pd.DataFrame(list(articles_with_abstracts.items()), columns=['PMID', 'Abstract'])
print(df)   # izpis dataframe
df.to_csv('izhod/pubmed_search_output.csv')   # shranimo kot datoteko CSV

       PMID                                           Abstract
0  33254559  The highly contagious coronavirus, which had a...
1  33195319  Coronavirus disease 2019 (COVID-19) is a rapid...
2  32826150  The coronavirus disease 2019 (COVID-19) pandem...
3  32244852  The epicenter of the original outbreak in Chin...
